In [ ]:
import os
import sys
import torch
import pandas as pd
import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mvbench_utils import get_mvbench, load_video

In [ ]:
frames = 10
MVBENCH_PATH = '<MVBENCH_PATH>'
VIDEO_PATH = f'<MVBENCH_PATH>/video'
SUBSET_PATH = '../subsets/mvbench.csv'

qa_data, qa_answers = get_mvbench(MVBENCH_PATH)

q_uids = pd.read_csv(SUBSET_PATH)['q_uid'].to_list()

if not os.path.exists(f"mvbench_features_{frames}"):
    os.mkdir(f"mvbench_features_{frames}")

In [ ]:
import math
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from extract.preprocessing import Preprocessing
from args import MODEL_DIR
import clip

In [ ]:
preprocess = Preprocessing()
model, _ = clip.load("ViT-L/14", download_root=MODEL_DIR)
model.eval()
model = model.cuda()

In [ ]:
with torch.no_grad():
    for q_uid in q_uids:
        video, timestamps = load_video(q_uid, qa_data[q_uid], frames, VIDEO_PATH, 224, 224)
        video = torch.from_numpy(video).permute((0, 3, 1, 2))
        video = preprocess(video)
        video = video.cuda()
        features = model.encode_image(video)
        features = features.cpu().numpy().astype('float16')
        np.save(f'mvbench_features_{frames}/{q_uid}.npy', features)